Generate copy number segments from PCAWG WGS data

In [53]:
import tarfile
import pandas as pd
import os
from pymongo import MongoClient

Testing scripts

In [5]:
fp = '../data/ICGC/CopyNumber/PBCA-DE/8685690a-d2e4-11e5-9065-4a655c6878de/6f4d836f-e86f-4b12-9549-7117f59e3d4a.svcp_1-0-2.20150115.somatic.cnv.tar.gz'
with tarfile.open(fp, 'r') as tar:
    for tarinfo in tar:
        if 'copynumber.txt' in tarinfo.name :
            f = tar.extractfile(tarinfo)
            lines = f.readlines()
            print(lines[3])
                
    

b'SNP_A-8497791\t1\t740857\t-0.350889181667628\t-0.136875445355132\t1\tNA\t2\t1\t0.63614582533042\n'


In [ ]:
i = 1
pre_value = 1000
pre_chro = '0'
segments = []

while(i<len(lines)):
    line = lines[i].decode("utf-8").split('\t')
    pos = line[2]
    cur_value = line[4]
    chro = line[1]
    
    if cur_value == pre_value:
        probes += 1
        end = pos
    else:
        if i>1:
            segments.append({'chro':pre_chro,'start':int(start),
                             'end':int(end),'probes':probes,'value':float(pre_value)})
        start = pos
        end = pos
        probes = 1
        pre_value = cur_value
        pre_chro = chro
        
    i += 1
    
segments.append({'chro':pre_chro,'start':int(start),
                 'end':int(end),'probes':probes,'value':float(pre_value)})       
    

            

Generate segment files from SVCP method, in PCAWG

In [6]:
def genPcawgSvcpSegments(filepath, outpath):
    
    # read the probe file from tar
    with tarfile.open(filepath, 'r') as tar:
        for tarinfo in tar:
            if 'copynumber.txt' in tarinfo.name :
                f = tar.extractfile(tarinfo)
                lines = f.readlines()

     
                # parse the probe file

                # init
                i = 1
                pre_value = 1000
                pre_chro = '0'
                segments = []

                while(i<len(lines)):
                    line = lines[i].decode("utf-8").split('\t')
                    pos = line[2]
                    cur_value = line[4]
                    chro = line[1]

                    if cur_value == pre_value:
                        probes += 1
                        end = pos
                    else:
                        if i>1:
                            segments.append({'chro':pre_chro,'start':int(start),
                                             'end':int(end),'probes':probes,'value':float(pre_value)})
                        start = pos
                        end = pos
                        probes = 1
                        pre_value = cur_value
                        pre_chro = chro

                    i += 1

                # add the last segment
                segments.append({'chro':pre_chro,'start':int(start),
                                 'end':int(end),'probes':probes,'value':float(pre_value)})    

                # output to file
                filename = os.path.basename(filepath)
                fileid = filename.split('.')[0]
                with open(outpath, 'w') as fo:
                    print('{}\t{}\t{}\t{}\t{}\t{}'.format('id','chro','start','end','probes','value'), file=fo)
                    for seg in segments:
                        print('{}\t{}\t{}\t{}\t{}\t{}'.format(fileid,seg['chro'],seg['start'],
                                                              seg['end'],seg['probes'],round(seg['value'],4)), file=fo)

In [7]:
walk_path = '/Volumes/originalData/ICGC/CopyNumber'
out_path = '../data'
for root, subdirs, files in os.walk(walk_path):
    for f in files:
        if 'somatic.cnv.tar.gz' in f and 'svcp' in f:
            fileid = f.split('.')[0]
            filename = '.'.join([fileid, 'cnv','tsv'])
            genPcawgSvcpSegments(os.path.join(root, f), os.path.join(out_path, filename))

Get meta data and save the combined info to db

In [70]:
# read pcawg_sample_sheet.tsv
pcawg_samples_sheet = pd.read_csv('../data/ICGC/Meta/pcawg_sample_sheet.tsv', sep='\t')

#read donor.all_projects.tsv
icgc_donors = pd.read_csv('../data/ICGC/Meta/donor.all_projects.tsv', sep='\t')

# #read pcawg_sample_sheet.2016-06-24.blacklisted_donors.tsv
# blacklist = pd.read_csv('/Volumes/originalData/ICGC/Meta/pcawg_sample_sheet.2016-06-24.blacklisted_donors.tsv', sep='\t')

pcawg_samples = []
for root, subdirs, files in os.walk(out_path):
    for f in files:
        fileid = f.split('.')[0]
        
        # sample info
        sample_info = pcawg_samples_sheet[(pcawg_samples_sheet['aliquot_id']==fileid) &
                                          (pcawg_samples_sheet['donor_wgs_exclusion_white_gray']=='Whitelist')]
        
        # blacklisted sample
        if len(sample_info) == 0:
            continue
            
        donorid = sample_info.iloc[0]['icgc_donor_id']
        specimen_type = sample_info.iloc[0]['dcc_specimen_type']
        specimen_type = specimen_type.split(' - ')
        sample_type = specimen_type[0]
        sample_source = specimen_type[1]
        
        
        # segments
        cnvs = []
        with open(os.path.join(root, f), 'r') as fin:
            next(fin)
            for line in fin:
                line = line.strip().split('\t')
                cnvs.append({'chro':line[1], 'start':int(line[2]), 'end':int(line[3]),
                             'probes':int(line[4]), 'value':float(line[5])})
                
        # donor info      
        donor = icgc_donors[icgc_donors['icgc_donor_id']==donorid]

        # blacklisted donor
        if len(donor) == 0:
            continue
        
        donor = donor.to_dict(orient='records')[0]
        donor['segments'] = cnvs
        donor['sample_type'] = sample_type
        donor['sample_source'] = sample_source
        donor['sample_id'] = fileid
        pcawg_samples.append(donor)
        

In [71]:
# write to db
db = MongoClient()['pcawg']['samples']
db.drop()
db.insert_many(pcawg_samples)

Tests

In [65]:
t = pcawg_samples_sheet[(pcawg_samples_sheet['dcc_project_code']=='PBCA-DE') &
                        (pcawg_samples_sheet['dcc_specimen_type']=='Primary tumour - solid tissue') &
                        (pcawg_samples_sheet['donor_wgs_exclusion_white_gray']=='Whitelist')]

In [69]:
len(pcawg_samples)

1896